# Reflection in LangGraph

- Author: [Heesun Moon](https://github.com/MoonHeesun)
- Peer Review:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/17-LangGraph/03-Use-Cases/16-LangGraph-Reflection.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/17-LangGraph/03-Use-Cases/16-LangGraph-Reflection.ipynb)

## Overview

Reflection in the context of LLM-based agents refers to the process of prompting an LLM to observe its past steps and evaluate the quality of its decisions. This is particularly useful in scenarios like iterative problem-solving, search refinement, and agent evaluation.

In this tutorial, we will explore **how to implement a simple Reflection mechanism using LangGraph**, specifically to analyze and improve **AI-generated essays**.

### What is Reflection?

Reflection involves prompting an LLM to analyze its own previous responses and adjust its future decisions accordingly.

This can be useful in:

- **Re-planning**: Improving the next steps based on past performance.

- **Search Optimization**: Refining retrieval strategies.

- **Evaluation**: Measuring the effectiveness of a solution and iterating.

We will implement a **Reflection-based agent in LangGraph** that reviews its own responses and refines them dynamically.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Defining the Reflection-Based Essay Generator](#defining-the-reflection-based-essay-generator)
- [Defining the Reflection Graph](#defining-the-reflection-graph)

### References

- [LangGraph Reflection](https://langchain-ai.github.io/langgraph/tutorials/reflection/reflection/)

----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain",
        "langgraph",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

You can set API keys in a `.env` file or set them manually.

[Note] If you’re not using the `.env` file, no worries! Just enter the keys directly in the cell below, and you’re good to go.

In [3]:
from dotenv import load_dotenv
from langchain_opentutorial import set_env

# Attempt to load environment variables from a .env file; if unsuccessful, set them manually.
if not load_dotenv():
    set_env(
        {
            "OPENAI_API_KEY": "",
            "LANGCHAIN_API_KEY": "",
            "LANGCHAIN_TRACING_V2": "true",
            "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        }
    )

# set the project name same as the title
set_env(
    {
        "LANGCHAIN_PROJECT": "LangGraph-Reflection",
    }
)

Environment variables have been set successfully.


## Defining the Reflection-Based Essay Generator

### 1. Generating the Essay

We will create a 5-paragraph essay generator that produces structured responses.

In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", streaming=True)
generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on how artificial intelligence is shaping the future of work and society"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

**Title: The Transformative Impact of Artificial Intelligence on the Future of Work and Society**

**Introduction**

Artificial Intelligence (AI) is no longer a concept confined to the realms of science fiction; it has become an integral part of our daily lives and is significantly shaping the future of work and society. As AI technologies continue to evolve, they are transforming industries, redefining job roles, and influencing social dynamics. This essay explores the multifaceted impact of AI on the workforce, the economy, and societal structures, highlighting both the opportunities and challenges that lie ahead.

**The Transformation of the Workforce**

One of the most profound effects of AI is its ability to automate tasks traditionally performed by humans. From manufacturing to customer service, AI systems are increasingly taking over repetitive and mundane tasks, allowing human workers to focus on more complex and creative endeavors. For instance, in the manufacturing sector, ro

### 2. Reflection

Now, we define the reflection prompt, where an AI evaluates the generated essay and suggests improvements.

In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [6]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

**Critique and Recommendations for Your Essay Submission**

Your essay titled "The Transformative Impact of Artificial Intelligence on the Future of Work and Society" presents a well-structured argument and covers a broad range of topics related to AI's influence. However, there are several areas where you can enhance the depth, clarity, and overall effectiveness of your writing. Below are detailed critiques and recommendations:

### Strengths:
1. **Clear Structure**: The essay is organized into distinct sections, making it easy to follow your argument. Each paragraph addresses a specific aspect of AI's impact.
2. **Relevant Examples**: You provide relevant examples, such as AI in manufacturing and healthcare, which help illustrate your points effectively.
3. **Balanced Perspective**: You acknowledge both the opportunities and challenges presented by AI, which adds depth to your analysis.

### Areas for Improvement:

1. **Length and Depth**:
   - **Recommendation**: Aim for a longer es

### 3. Iterative Improvement

We can repeat the process, incorporating feedback into new essay versions.

In [7]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

**Title: The Transformative Impact of Artificial Intelligence on the Future of Work and Society**

**Introduction**

As we stand on the brink of a technological revolution, Artificial Intelligence (AI) is emerging as a pivotal force reshaping the landscape of work and society. With the potential to revolutionize industries, enhance productivity, and redefine human roles, AI is not merely a tool but a transformative agent that challenges our traditional notions of employment and social interaction. According to a recent report by the World Economic Forum, over 85 million jobs may be displaced by 2025 due to the rise of AI, while simultaneously, 97 million new roles could emerge. This essay delves into the multifaceted impact of AI on the workforce, the economy, and societal structures, highlighting both the opportunities and challenges that lie ahead.

**The Transformation of the Workforce**

AI's most significant impact is arguably its ability to automate tasks that were once the domai

## Defining the Reflection Graph

Now that we've implemented each step, we wire everything into a LangGraph workflow.

### 1. Define State

We establish a structured way to store and track messages. This ensures that each interaction, including generated essays and reflections, is retained in a structured manner for iterative improvements.

In [8]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]

### 2. Create Nodes

Two key nodes are defined—one for generating essays and another for evaluating them. The `generation_node` creates a structured essay, while the `reflection_node` critiques and suggests improvements.

In [9]:
async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}

In [10]:
async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}

### 3. Set Conditional Loops

The workflow continues refining until a stopping criterion is met. In this case, after a few iterations, the graph stops the refinement process to prevent infinite loops.

In [11]:
def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"

### 4. Compile and Execute

The LangGraph structure is compiled, allowing seamless AI-driven iteration. The process begins with the `generate` node and loops through the `reflect` node until the stopping condition is met.

In [12]:
builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")
builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

### 5. Execute the Graph

The graph is now executed in an asynchronous streaming process, where an essay is generated, reflected upon, and improved iteratively.

In [13]:
config = {"configurable": {"thread_id": "1"}}

In [14]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the ethical implications of artificial intelligence in decision-making"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='**Title: The Ethical Implications of Artificial Intelligence in Decision-Making**\n\nIn recent years, artificial intelligence (AI) has emerged as a transformative force across various sectors, from healthcare to finance, and even in governance. As AI systems increasingly take on decision-making roles traditionally held by humans, the ethical implications of this shift have become a focal point of discussion. This essay explores the ethical concerns surrounding AI in decision-making, including issues of bias, accountability, transparency, and the potential for dehumanization.\n\nOne of the most pressing ethical concerns regarding AI in decision-making is the issue of bias. AI systems are trained on vast datasets that often reflect historical inequalities and prejudices. When these biased datasets are used to train AI algorithms, the resulting systems can perpetuate and even exacerbate existing social injustices. For instance, in the criminal

### 6. Retrieve Final State

After execution, we retrieve the final state of messages, including the refined essay and reflections.

In [15]:
state = graph.get_state(config)

In [16]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Generate an essay on the ethical implications of artificial intelligence in decision-making

================================== Ai Message ==================================

**Title: The Ethical Implications of Artificial Intelligence in Decision-Making**

In recent years, artificial intelligence (AI) has emerged as a transformative force across various sectors, from healthcare to finance, and even in governance. As AI systems increasingly take on decision-making roles traditionally held by humans, the ethical implications of this shift have become a focal point of discussion. This essay explores the ethical concerns surrounding AI in decision-making, including issues of bias, accountability, transparency, and the potential for dehumanization.

One of the most pressing ethical concerns regarding AI in decision-making is the issue of bias. AI systems are trained on vast datasets that often reflect histori